In [1]:
import pandas as pd
import numpy as np
import math
import statsmodels.formula.api as sm

In [2]:
#导入数据
m_ret = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\CRSP_common_stock_monthly.csv")
beta  = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\Result_monthly_beta.csv")
FF_Rf = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\FF_monthly_Rf.csv")
BAB  = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\Result_BAB_factor.csv")
FF4  = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\FF_4factor.csv")
Liquid_factor = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\liquidity factor.csv")

In [3]:
#首先处理无风险数据
FF_Rf['year']=(FF_Rf['dateff']/10000).apply(int)
FF_Rf['month']=(FF_Rf['dateff']/100).apply(int)-FF_Rf['year']*100
FF_Rf.drop(columns=['dateff'],inplace=True)

In [4]:
#然后处理common stock数据，分离年份和月份
m_ret['year']=(m_ret['date']/10000).apply(int)
m_ret['month']=(m_ret['date']/100).apply(int)-m_ret['year']*100
m_ret.rename(columns={'PERMNO':'id'},inplace=True)
m_ret.drop(columns=['date','SHRCD'],inplace=True)
m_ret['RET'].replace(['B','C'],[np.nan,np.nan],inplace=True)
m_ret['RET']=m_ret['RET'].apply(float)

In [5]:
#合并m_ret和FF_Rf
stock=pd.merge(m_ret, FF_Rf, on=['year','month'], how='left')
stock=pd.merge(stock,beta,on=['id','year','month'])
del m_ret
del FF_Rf
del beta

In [6]:
#求stock的超额收益
stock['ret']=stock['RET']-stock['rf']
stock.drop(columns=['RET','rf'],inplace=True)

In [7]:
#计算BAB的超额收益
BAB['date']=BAB['year']*100+BAB['month']
BAB.drop(columns=['year','month'],inplace=True)
BAB=BAB[BAB['date']>=192904]

In [8]:
Excess_BAB=BAB['BAB'].mean()
Volatility_BAB = BAB["BAB"].var()
SharpeRatio_BAB = Excess_BAB / np.sqrt(Volatility_BAB)
print("BAB portfolio's excess return is: {:.2f}%".format(Excess_BAB*100))
print("BAB portfolio's volatility is: {:.2f}%".format(Volatility_BAB*10000))
print("BAB portfolio's sharpe ratio is: {:.2f}".format(SharpeRatio_BAB))
summary = {}
summary["Excess_return"] = round(Excess_BAB*100,2)

BAB portfolio's excess return is: 0.72%
BAB portfolio's volatility is: 11.30%
BAB portfolio's sharpe ratio is: 0.22


In [9]:
# n-factor model, n = 1,3,4,5
## 首先处理FF4的数据
FF4['date']=(FF4['dateff']/100).apply(int)
FF4.drop(columns=['dateff'],inplace=True)

In [10]:
BAB = pd.merge(BAB,FF4,on = "date", how = "left")

In [12]:
## CPAM alpha
result = sm.ols(formula = "BAB ~ 1 + mktrf",data = BAB).fit()
print("BAB portfolio's CAPM alpha is: {:.2f}".format(result.params["Intercept"]*100))
summary["CAPM_alpha"] = round(result.params["Intercept"]*100,2)

BAB portfolio's CAPM alpha is: 0.75


In [13]:
## Three-factor alpha
result = sm.ols(formula = "BAB ~ 1 + mktrf+ smb+ hml",data = BAB).fit()
print("BAB portfolio's three-factor alpha is: {:.2f}".format(result.params["Intercept"]*100))
summary["three_factor_alpha"] = round(result.params["Intercept"]*100,2)

BAB portfolio's three-factor alpha is: 0.76


In [14]:
## Four-factor alpha
result = sm.ols(formula = "BAB ~ 1 + mktrf+ smb+ hml+ umd",data = BAB).fit()
print("BAB portfolio's four-factor alpha is: {:.2f}".format(result.params["Intercept"]*100))
summary["four_factor_alpha"] = round(result.params["Intercept"]*100,2)

BAB portfolio's four-factor alpha is: 0.53


In [15]:
## Five-factor alpha
Liquid_factor = Liquid_factor[["DATE","PS_VWF"]]
Liquid_factor = Liquid_factor.rename(columns = {"DATE":"date","PS_VWF":"vwf"})
Liquid_factor["date"] = (Liquid_factor['date']/100).apply(int)
BAB = pd.merge(BAB,Liquid_factor,on = "date", how = "left")

In [16]:
result = sm.ols(formula = "BAB ~ 1 + mktrf+ smb+ hml+ umd+ vwf",data = BAB,missing='drop').fit()
print("BAB portfolio's five-factor alpha is: {:.2f}".format(result.params["Intercept"]*100))
summary["five_factor_alpha"] = round(result.params["Intercept"]*100,2)

BAB portfolio's five-factor alpha is: 0.51


In [17]:
summary["Volatility"] = round(Volatility_BAB * 10000,3)
summary["SharpeRatio"] = round(SharpeRatio_BAB,2)

In [23]:
pd.DataFrame(summary,index = ["BAB"]).T

,BAB
Excess_return,0.720
CAPM_alpha,0.750
three_factor_alpha,0.760
four_factor_alpha,0.530
five_factor_alpha,0.510
Volatility,11.303
SharpeRatio,0.220
